In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
# Read the training and test data
train_df = pd.read_json("../input/train.json")
test_df = pd.read_json("../input/test.json")

#Look at the size of test and train data
print("train data shape: ", train_df.shape[0]);
print("test data shape: ", test_df.shape[0]);

In [ ]:
def string2numeric_hash(text):
    import hashlib
    return int(hashlib.md5(text).hexdigest()[:8], 16)

In [ ]:
# Convert the features like features, photos, description into numeric by computing their length
# Generate hash for the building_id, manager_id
train_df["num_photos"] = train_df["photos"].apply(len)
train_df["num_features"] = train_df["features"].apply(len)
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
train_df["building_gen_id"] = train_df["building_id"].apply(lambda x: x.encode('utf-8'))
train_df["building_gen_id"] = train_df["building_gen_id"].apply(lambda x: string2numeric_hash(x))
train_df["manager_gen_id"] = train_df["manager_id"].apply(lambda x: x.encode('utf-8'))
train_df["manager_gen_id"] = train_df["manager_gen_id"].apply(lambda x: string2numeric_hash(x))


test_df["num_photos"] = test_df["photos"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))
test_df["building_gen_id"] = test_df["building_id"].apply(lambda x: x.encode('utf-8'))
test_df["building_gen_id"] = test_df["building_gen_id"].apply(lambda x: string2numeric_hash(x))
test_df["manager_gen_id"] = test_df["manager_id"].apply(lambda x: x.encode('utf-8'))
test_df["manager_gen_id"] = test_df["manager_gen_id"].apply(lambda x: string2numeric_hash(x))

In [ ]:
# Select the features and prepare the Input and target variables
selected_features = ["bedrooms", "bathrooms", "price", "num_photos", "num_features", "num_description_words", "building_gen_id", "manager_gen_id"]

X = train_df[selected_features]
Y = train_df["interest_level"]

X.head()

In [ ]:
#Split the input into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.33)
#Pass the input to the algo and calculate the loss
from sklearn.ensemble import RandomForestClassifier
algo = RandomForestClassifier(n_estimators=100)
algo.fit(X_train, Y_train)
y_predict_val = algo.predict_proba(X_val)
from sklearn.metrics import log_loss
log_loss(Y_val, y_predict_val)

In [ ]:
X_test = test_df[selected_features]
y_predict_test = algo.predict_proba(X_test)

In [ ]:
labels2idx = {label: i for i, label in enumerate(algo.classes_)}
labels2idx

In [ ]:
sub = pd.DataFrame()
sub["listing_id"] = test_df["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_predict_test[:, labels2idx[label]]
sub.to_csv("categoral_numeric.csv", index=False)
print("process is done1")

**Public Score: 1.29721**